In [2]:
#Imports
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
#Read in Data
red_df = pd.read_csv('Resources/winequality-red.csv', delimiter=';')
white_df = pd.read_csv('Resources/winequality-white.csv', delimiter=';')

In [24]:
def TestModel(features_to_test):
    #Splitting
    target_red = pd.get_dummies(red_df['quality'])
    features_red = red_df[features_to_test]

    target_white = pd.get_dummies(white_df['quality'])
    features_white = white_df[features_to_test]

    X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(features_red, target_red, random_state=42)
    X_train_white, X_test_white, y_train_white, y_test_white = train_test_split(features_white, target_white, random_state=42)

    #Scaling
    X_scaler_red = skl.preprocessing.StandardScaler()
    X_scaler_white = skl.preprocessing.StandardScaler()

    X_scaler_red.fit(X_train_red)
    X_scaler_white.fit(X_train_white)

    X_train_scaled_red = X_scaler_red.transform(X_train_red)
    X_test_scaled_red = X_scaler_red.transform(X_test_red)

    X_train_scaled_white = X_scaler_white.transform(X_train_white)
    X_test_scaled_white = X_scaler_white.transform(X_test_white)

    inputs = len(features_to_test)

    #Create and Evaluate Red Model
    model_red = tf.keras.models.Sequential()
    model_red.add(tf.keras.layers.Dense(units=25, activation='selu', input_dim=inputs))
    model_red.add(tf.keras.layers.Dense(units=20, activation='softplus'))
    model_red.add(tf.keras.layers.Dense(units=10, activation='softsign'))
    model_red.add(tf.keras.layers.Dense(len(set(red_df['quality'].values)), activation='softmax'))
    
    model_red.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    fit_model_red = model_red.fit(X_train_scaled_red, y_train_red, epochs=30)
    model_red_loss, model_red_accuracy = model_red.evaluate(X_test_scaled_red, y_test_red, verbose=2)
    
    #Create and Evaluate White Model
    model_white = tf.keras.models.Sequential()
    model_white.add(tf.keras.layers.Dense(units=25, activation='selu', input_dim=inputs))
    model_white.add(tf.keras.layers.Dense(units=20, activation='softplus'))
    model_white.add(tf.keras.layers.Dense(units=10, activation='softsign'))
    model_white.add(tf.keras.layers.Dense(len(set(red_df['quality'].values)), activation='softmax'))

    model_white.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    fit_model_white = model_white.fit(X_train_scaled_white, y_train_white, epochs=25)
    model_white_loss, model_white_accuracy = model_white.evaluate(X_test_scaled_white, y_test_white, verbose=2)
   
    return({'Red_Loss':model_red_loss, 'Red_Acc':model_red_accuracy, 
            'White_Loss':model_white_loss, 'White_Acc':model_white_accuracy})
    

In [21]:
    model_red = tf.keras.models.load_model('Resources/model.h5')
    model_red.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 21)                252       
                                                                 
 dense_1 (Dense)             (None, 17)                374       
                                                                 
 dense_2 (Dense)             (None, 5)                 90        
                                                                 
 dense_3 (Dense)             (None, 11)                66        
                                                                 
 dense_4 (Dense)             (None, 6)                 72        
                                                                 
Total params: 854 (3.34 KB)
Trainable params: 854 (3.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
feature_results = []

In [14]:
#Test Single Features
cols = red_df.drop(columns='quality').columns
results = []
for feature in cols:
    result = TestModel(feature)
    results.append({'feature':feature, 'result':result})
feature_results.append({'single_features':results})

ValueError: Expected 2D array, got 1D array instead:
array=[11.7  8.8  7.1 ...  7.2  7.9  5.8].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [25]:
#Test Double Features
cols = red_df.drop(columns='quality').columns
results = []
for feature_a in cols:
    for feature_b in cols:
        result = TestModel([feature_a, feature_b])
        results.append({'features': [feature_a, feature_b], 'result':result})
feature_results.append({'double_features':results})

AttributeError: Can't set the attribute "input_shape", likely because it conflicts with an existing read-only @property of the object. Please choose a different name.

In [ ]:
#Test Triple Features
cols = red_df.drop(columns='quality').columns
results = []
for feature_a in cols:
    for feature_b in cols:
        for feature_c in cols:

            result = TestModel([feature_a, feature_b])
            results.append({'features': [feature_a, feature_b], 'result':result})